In [1]:
# Cell 1: Setup & Environment Installation

# --- ติดตั้ง Libraries ---
# segmentation-models-pytorch: คลังโมเดล U-Net, FPN และอื่นๆ พร้อม Loss เฉพาะทาง
# albumentations: สำหรับทำ Data Augmentation ที่มีประสิทธิภาพสำหรับ Segmentation
# opendatasets: สำหรับดาวน์โหลดข้อมูลจาก Kaggle
!pip install segmentation-models-pytorch albumentations opendatasets --quiet

# --- Import Libraries ที่จำเป็น ---
import os
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt

print("Setup Complete. Libraries are ready.")


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Setup Complete. Libraries are ready.


In [2]:
# Cell 2: Data Acquisition
import opendatasets as od

dataset_url = 'https://www.kaggle.com/c/hubmap-kidney-segmentation'
od.download(dataset_url)

# --- กำหนด Path หลัก ---
DATA_DIR = './hubmap-kidney-segmentation/'
TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train')
TEST_IMG_DIR = os.path.join(DATA_DIR, 'test')
TRAIN_CSV_PATH = os.path.join(DATA_DIR, 'train.csv')

print(f"Data downloaded to: {DATA_DIR}")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:

KeyboardInterrupt: 

In [ ]:
# Cell 3: Run-Length Encoding (RLE) Utility Functions

# ฟังก์ชันแปลง RLE string เป็น 2D Mask
def rle2mask(rle_string, shape=(512, 512)):
    if pd.isna(rle_string):
        return np.zeros(shape).astype(np.uint8)
        
    s = rle_string.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T # ต้อง Transpose

# ฟังก์ชันแปลง 2D Mask เป็น RLE string
def mask2rle(img):
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# --- ทดลองการทำงาน ---
df = pd.read_csv(TRAIN_CSV_PATH)
sample_rle = df['encoding'].iloc[0]
sample_mask = rle2mask(sample_rle, shape=(df['img_height'][0], df['img_width'][0]))

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(sample_mask, cmap='gray')
plt.title("Mask from RLE")
plt.axis('off')

# ลองแปลงกลับ
reconverted_rle = mask2rle(sample_mask)
print(f"Original RLE length: {len(sample_rle)}")
print(f"Reconverted RLE length: {len(reconverted_rle)}")
print("Lengths should be identical.")


In [ ]:
# Cell 4: Custom PyTorch Dataset for Segmentation

class HuBMAPDataset(Dataset):
    def __init__(self, df, img_dir, transforms=None):
        self.df = df
        self.img_dir = img_dir
        self.image_ids = df['id'].unique()
        self.transforms = transforms

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        
        # โหลดภาพ
        img_path = os.path.join(self.img_dir, f"{image_id}.tiff")
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # สร้าง Mask
        rle_string = self.df[self.df['id'] == image_id]['encoding'].iloc[0]
        mask = rle2mask(rle_string, (image.shape[0], image.shape[1]))
        
        # ทำ Augmentation
        if self.transforms:
            transformed = self.transforms(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']
        
        # เพิ่มมิติของ channel ให้ mask (H, W) -> (1, H, W)
        mask = mask.unsqueeze(0)
        
        return image, mask

# --- สร้าง Data Augmentation Pipeline ---
IMG_SIZE = 256

train_transforms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.3),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transforms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])


In [ ]:
# Cell 5: Create DataLoaders

from sklearn.model_selection import train_test_split

# แบ่งข้อมูล
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# สร้าง Datasets
train_dataset = HuBMAPDataset(train_df, TRAIN_IMG_DIR, transforms=train_transforms)
val_dataset = HuBMAPDataset(val_df, TRAIN_IMG_DIR, transforms=val_transforms)

# สร้าง DataLoaders
BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")

# --- ตรวจสอบขนาดของข้อมูล ---
images, masks = next(iter(train_loader))
print(f"Image batch shape: {images.shape}") # (Batch, Channels, Height, Width)
print(f"Mask batch shape: {masks.shape}")   # (Batch, 1, Height, Width)


In [ ]:
# Cell 6: Define Model, Loss, and Optimizer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# สร้างโมเดล U-Net
# activation='sigmoid' เพราะเราทำ binary segmentation (เป็นเนื้องอกหรือไม่เป็น)
model = smp.Unet(
    encoder_name="resnet34",        # backbone
    encoder_weights="imagenet",     # ใช้ pre-trained weights
    in_channels=3,                  # ภาพสี RGB
    classes=1,                      # แค่ 1 คลาส (glomerulus)
    activation='sigmoid'
).to(DEVICE)

# กำหนด Loss Function และ Optimizer
# DiceLoss เหมาะกับ Imbalanced data ใน segmentation
loss_fn = smp.losses.DiceLoss(mode='binary')
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

print(f"Model loaded on {DEVICE}")


In [ ]:
# Cell 7: The Training Loop

# Metric ที่จะใช้วัดผล (Intersection over Union และ Dice Score)
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
    smp.utils.metrics.F1Score(threshold=0.5) # Same as Dice Score
]

EPOCHS = 10 # โจทย์จริงอาจใช้ 20-50
best_dice_score = 0.0
BEST_MODEL_PATH = "best_model.pth"

# วนลูปการเทรน
for epoch in range(EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{EPOCHS} ---")
    
    # --- Training Phase ---
    model.train()
    train_loss = 0
    for images, masks in tqdm(train_loader, desc="Training"):
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        
        optimizer.zero_grad()
        preds = model(images)
        loss = loss_fn(preds, masks)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    # --- Validation Phase ---
    model.eval()
    val_loss = 0
    val_iou = 0
    val_dice = 0
    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc="Validating"):
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            preds = model(images)
            val_loss += loss_fn(preds, masks).item()
            
            # คำนวณ metrics
            tp, fp, fn, tn = smp.metrics.get_stats(preds.round(), masks.long(), mode='binary')
            val_iou += smp.metrics.iou_score(tp, fp, fn, tn, reduction='micro')
            val_dice += smp.metrics.f1_score(tp, fp, fn, tn, reduction='micro')
    
    # คำนวณค่าเฉลี่ย
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    avg_val_iou = val_iou / len(val_loader)
    avg_val_dice = val_dice / len(val_loader)
    
    print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val IoU: {avg_val_iou:.4f} | Val Dice: {avg_val_dice:.4f}")

    # บันทึกโมเดลที่ดีที่สุด
    if avg_val_dice > best_dice_score:
        print(f"Dice score improved from {best_dice_score:.4f} to {avg_val_dice:.4f}. Saving model...")
        best_dice_score = avg_val_dice
        torch.save(model.state_dict(), BEST_MODEL_PATH)

print("\nFinished Training!")


In [ ]:
# Cell 8: Visualize Predictions

# โหลดโมเดลที่ดีที่สุด
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.eval()

# เลือกภาพตัวอย่างจาก validation set
sample_img, sample_mask = val_dataset[np.random.randint(len(val_dataset))]

# ทำนายผล
with torch.no_grad():
    pred_mask = model(sample_img.unsqueeze(0).to(DEVICE))
    pred_mask = pred_mask.squeeze().cpu().numpy()

# แปลง tensor กลับเป็นภาพที่แสดงผลได้
img_display = sample_img.permute(1, 2, 0).numpy() # C,H,W -> H,W,C
mask_display = sample_mask.squeeze().numpy()
pred_mask_display = (pred_mask > 0.5).astype(np.uint8) # Thresholding

# แสดงผล
fig, ax = plt.subplots(1, 3, figsize=(18, 6))
ax[0].imshow(img_display)
ax[0].set_title("Original Image")
ax[0].axis('off')

ax[1].imshow(mask_display, cmap='gray')
ax[1].set_title("Ground Truth Mask")
ax[1].axis('off')

ax[2].imshow(pred_mask_display, cmap='gray')
ax[2].set_title("Predicted Mask")
ax[2].axis('off')

plt.show()


In [ ]:
# Cell 9: Generate Submission File
submission_df = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))
test_ids = submission_df['id'].tolist()

predictions = []

# ใช้ val_transforms กับ test set
test_transforms = val_transforms

# วนลูปทำนายผลใน test set
model.eval()
with torch.no_grad():
    for img_id in tqdm(test_ids, desc="Generating Submission"):
        img_path = os.path.join(TEST_IMG_DIR, f"{img_id}.tiff")
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_shape = (image.shape[0], image.shape[1])
        
        # Preprocess
        transformed = test_transforms(image=image)
        image_tensor = transformed['image'].unsqueeze(0).to(DEVICE)
        
        # Predict
        pred_mask_tensor = model(image_tensor)
        
        # Post-process
        pred_mask_resized = nn.functional.interpolate(
            pred_mask_tensor, 
            size=original_shape, 
            mode='bilinear', 
            align_corners=False
        )
        pred_mask = (pred_mask_resized.squeeze().cpu().numpy() > 0.5).astype(np.uint8)
        
        # Convert to RLE
        rle = mask2rle(pred_mask)
        predictions.append(rle)

# สร้างไฟล์ CSV
submission_df['rle'] = predictions
submission_df.to_csv('submission.csv', index=False)

print("\nsubmission.csv created successfully!")
display(submission_df.head())
